In [21]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import collections
from torch.utils.data import random_split
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# Cấu hình thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. Xây dựng Vocabulary ---
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    def build_vocabulary(self, sentence_list):
        frequencies = collections.Counter()
        idx = 4
        
        for sentence in sentence_list:
            for word in sentence.lower().split():
                frequencies[word] += 1
                
        for word, count in frequencies.items():
            if count >= self.freq_threshold:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1

    def numericalize(self, text):
        tokenized_text = text.lower().split()
        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

# --- 2. Xây dựng Dataset ---
class Flickr8kDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform
        
        # Lấy captions và ảnh
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]
        
        # Xây dựng vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)

# --- 3. Collate Function (Padding) ---
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=True, padding_value=self.pad_idx)
        return imgs, targets

# --- Thiết lập Path và Loader ---
# Lưu ý: Cấu trúc thư mục trên Kaggle thường là /kaggle/input/flickr8k/Images và captions.txt
image_folder = '/kaggle/input/flickr8k/Images'
captions_file = '/kaggle/input/flickr8k/captions.txt'

transform = transforms.Compose([
    transforms.Resize((224, 224)), # ResNet chuẩn input 224
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load dữ liệu (Cần đảm bảo file tồn tại, nếu không code sẽ báo lỗi)
try:
    dataset = Flickr8kDataset(image_folder, captions_file, transform=transform)
    pad_idx = dataset.vocab.stoi["<PAD>"]
    # train_loader = DataLoader(
    #     dataset=dataset,
    #     batch_size=32, # Batch size theo yêu cầu
    #     num_workers=2,
    #     shuffle=True,
    #     collate_fn=MyCollate(pad_idx=pad_idx)
    # )

    # 1. Xác định kích thước tập train và test (ví dụ: 90% train, 10% test)
    train_size = int(0.9 * len(dataset))
    test_size = len(dataset) - train_size
    
    # 2. Chia ngẫu nhiên dataset
    train_set, test_set = random_split(dataset, [train_size, test_size])
    
    # 3. Tạo lại Train Loader (nếu muốn train trên đúng tập train_set mới)
    train_loader = DataLoader(
        dataset=train_set,
        batch_size=32,
        num_workers=2,
        shuffle=True,
        collate_fn=MyCollate(pad_idx=pad_idx)
    )
    
    # 4. Tạo Test Loader (đây là biến bạn đang thiếu)
    test_loader = DataLoader(
        dataset=test_set,
        batch_size=32,
        num_workers=2,
        shuffle=False, # Test không cần shuffle
        collate_fn=MyCollate(pad_idx=pad_idx)
    )

    print(f"Train size: {len(train_set)}, Test size: {len(test_set)}")
    print(f"Vocab size: {len(dataset.vocab)}")
except Exception as e:
    print(f"Lỗi load data: {e}. Hãy kiểm tra lại đường dẫn file trên Kaggle.")

Using device: cuda
Train size: 36409, Test size: 4046
Vocab size: 3005


In [22]:
import torchvision.models as models

import torch
import torch.nn as nn
import torchvision
from torchvision.models import ViT_B_16_Weights

class ViTEncoder(nn.Module):
    def __init__(self, out_dim=512):
        super().__init__()
        # Load pre-trained ViT-B/16
        weights = ViT_B_16_Weights.DEFAULT
        self.vit = torchvision.models.vit_b_16(weights=weights)
        
        # ViT-B/16 có hidden_dim = 768. 
        # Ta cần chiếu về out_dim (512) để khớp với Decoder LSTM ở Module 1
        self.proj = nn.Linear(768, out_dim)
        
        # Freeze các tầng đầu của ViT để train nhanh hơn (tùy chọn)
        for param in self.vit.parameters():
            param.requires_grad = False 
        # Unfreeze projection layer
        for param in self.proj.parameters():
            param.requires_grad = True

    def forward(self, x):
        # x: B x 3 x 224 x 224
        
        # 1. Chuyển ảnh thành patch embeddings
        # Hàm _process_input thực hiện: Conv2d (chia patch) -> Flatten -> Transpose
        x = self.vit._process_input(x) 
        n = x.shape[0]

        # 2. Thêm Class Token (Learnable)
        batch_class_token = self.vit.class_token.expand(n, -1, -1)
        x = torch.cat([batch_class_token, x], dim=1) # B x (1 + 196) x 768 (với ảnh 224x224, patch 16 -> 14x14=196 patches)

        # 3. Cộng Positional Embedding và đi qua Encoder Layers
        x = self.vit.encoder(x) # B x 197 x 768

        # 4. Chiếu về chiều dữ liệu mong muốn
        x = self.proj(x) # B x 197 x 512

        # Tách CLS token và Patch tokens
        cls_token = x[:, 0]     # Global Vector (B x 512)
        patch_tokens = x[:, 1:] # Memory cho Attention (B x 196 x 512)
        
        return cls_token, patch_tokens

# --- 1. Positional Encoding ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        # Tạo ma trận PE
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        
        # Công thức: div_term = 10000^(2i/d_model)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        
        # Thêm batch dimension: 1 x max_len x D
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        # x: B x L x D
        L = x.size(1)
        # Cộng PE vào embedding, slice theo độ dài thực tế
        return x + self.pe[:, :L, :]

# --- 2. Transformer Decoder ---
class TransformerCaptionDecoder(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=6, max_len=100):
        super().__init__()
        
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos = PositionalEncoding(d_model, max_len=max_len)
        
        # Decoder Layer chuẩn của PyTorch
        # batch_first=True giúp input/output có dạng [Batch, Seq, Dim]
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=2048, # Thường gấp 4 lần d_model
            dropout=0.1,
            batch_first=True 
        )
        
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, vocab_size)
        self.d_model = d_model

    def generate_square_subsequent_mask(self, sz, device):
        # Tạo mask che phần tương lai (triangular mask)
        # Giá trị -inf để softmax về 0, hoặc True/False tuỳ version
        mask = torch.triu(torch.ones(sz, sz, device=device) * float('-inf'), diagonal=1)
        return mask

    def forward(self, captions, memory, tgt_padding_mask=None):
        # captions: B x T (Target input)
        # memory: B x L x D (Image features from CNN)
        
        # 1. Embedding + Positional Encoding
        tgt = self.emb(captions) * math.sqrt(self.d_model) # Scale embedding
        tgt = self.pos(tgt)
        
        # 2. Tạo Causal Mask (Che tương lai)
        T = tgt.size(1)
        tgt_mask = self.generate_square_subsequent_mask(T, tgt.device)
        
        # 3. Transformer Decoder Pass
        # memory_key_padding_mask=None vì ảnh có kích thước cố định (49 vị trí)
        output = self.decoder(
            tgt=tgt, 
            memory=memory, 
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_padding_mask 
        )
        
        # 4. Dự đoán từ
        logits = self.fc(output) # B x T x Vocab
        return logits
        

class TransformerImageCaptioning(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, images, captions):
        _, memory = self.encoder(images) # Memory: B x 49 x 512
        tgt_padding_mask = (captions == 0) # Giả sử <PAD> index là 0
        logits = self.decoder(captions, memory, tgt_padding_mask=tgt_padding_mask)
        
        return logits
        return outputs

In [23]:
# --- Cấu hình Hyperparameters ---
# vocab_size lấy từ dataset Module 1
d_model = 512 
nhead = 8
num_layers = 4 

# Khởi tạo
encoder = CNNEncoder(out_dim=d_model).to(device) # Sử dụng CNNEncoder từ Module 1
decoder = TransformerCaptionDecoder(vocab_size, d_model, nhead, num_layers).to(device)
model = ViTLSTMVgModel(encoder, decoder).to(device)

import torch.optim as optim

# --- Training Configuration ---
criterion = nn.CrossEntropyLoss(ignore_index=0) # 0 is padding idx

# Learning rate phân tầng (Differential Learning Rate)
# ViT đã pre-train nên để LR rất nhỏ, Decoder train từ đầu nên để LR cao hơn
optimizer = optim.Adam([
    {'params': model.encoder.parameters(), 'lr': 1e-5},
    {'params': model.decoder.parameters(), 'lr': 1e-4}
])

# --- Training Loop ---
print("Bắt đầu huấn luyện ViT + Transformer Decoder...")
model.train()

num_epochs = 3 
for epoch in range(num_epochs):
    epoch_loss = 0
    for idx, (imgs, captions) in enumerate(train_loader):
        imgs = imgs.to(device)
        captions = captions.to(device)
        
        # Decoder input: bỏ token cuối
        decoder_input = captions[:, :-1]
        # Target: bỏ token đầu <SOS>
        targets = captions[:, 1:]
        
        # Forward pass
        # Transformer tự xử lý masking bên trong
        outputs = model(imgs, decoder_input)
        
        # Flatten and Loss
        loss = criterion(outputs.reshape(-1, vocab_size), targets.reshape(-1))
        
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Clipping (Rất quan trọng với Transformer để tránh nan loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        epoch_loss += loss.item()
        
        if idx % 50 == 0:
            print(f"Epoch {epoch+1}, Step {idx}, Loss: {loss.item():.4f}")
            
    print(f"End Epoch {epoch+1}, Avg Loss: {epoch_loss/len(train_loader):.4f}")

Bắt đầu huấn luyện ViT + Transformer Decoder...
Epoch 1, Step 0, Loss: 8.1995
Epoch 1, Step 50, Loss: 4.5011
Epoch 1, Step 100, Loss: 4.2636
Epoch 1, Step 150, Loss: 3.8227
Epoch 1, Step 200, Loss: 3.8478
Epoch 1, Step 250, Loss: 3.5585
Epoch 1, Step 300, Loss: 3.5896
Epoch 1, Step 350, Loss: 3.5196
Epoch 1, Step 400, Loss: 3.3648
Epoch 1, Step 450, Loss: 3.2477
Epoch 1, Step 500, Loss: 3.2910
Epoch 1, Step 550, Loss: 3.2742
Epoch 1, Step 600, Loss: 3.4358
Epoch 1, Step 650, Loss: 3.2104
Epoch 1, Step 700, Loss: 3.3444
Epoch 1, Step 750, Loss: 3.1201
Epoch 1, Step 800, Loss: 3.0014
Epoch 1, Step 850, Loss: 2.7298
Epoch 1, Step 900, Loss: 3.0843
Epoch 1, Step 950, Loss: 2.6566
Epoch 1, Step 1000, Loss: 2.9127
Epoch 1, Step 1050, Loss: 3.0958
Epoch 1, Step 1100, Loss: 2.8712
End Epoch 1, Avg Loss: 3.4323
Epoch 2, Step 0, Loss: 2.8865
Epoch 2, Step 50, Loss: 2.8317
Epoch 2, Step 100, Loss: 2.7789
Epoch 2, Step 150, Loss: 2.9493
Epoch 2, Step 200, Loss: 2.8489
Epoch 2, Step 250, Loss: 2.93

In [24]:
# from torchtext.data.metrics import bleu_score
# Hoặc dùng nltk nếu torchtext bản mới đổi API:
from nltk.translate.bleu_score import corpus_bleu

def evaluate_bleu(model, dataloader, device, vocab):
    model.eval()
    refs = []  # Danh sách caption gốc
    hyps = []  # Danh sách caption dự đoán
    
    with torch.no_grad():
        for idx, (imgs, captions) in enumerate(dataloader):
            if idx > 20: break # Test nhanh trên 20 batch đầu
            imgs = imgs.to(device)
            
            # Encoder
            _, memory = model.encoder(imgs)
            
            # Autoregressive Generation
            # Bắt đầu với <SOS>
            batch_size = imgs.size(0)
            ys = torch.ones(batch_size, 1).fill_(vocab.stoi["<SOS>"]).type(torch.long).to(device)
            
            for i in range(20): # Max length caption
                tgt_mask = model.decoder.generate_square_subsequent_mask(ys.size(1), device)
                
                # Gọi decoder với chuỗi hiện tại ys
                out = model.decoder.decoder(
                    model.decoder.pos(model.decoder.emb(ys) * math.sqrt(d_model)), 
                    memory, 
                    tgt_mask=tgt_mask
                )
                
                # Lấy logit của từ cuối cùng
                prob = model.decoder.fc(out[:, -1])
                _, next_word = torch.max(prob, dim=1)
                
                ys = torch.cat([ys, next_word.unsqueeze(1)], dim=1)
            
            # Convert IDs to Words
            for j in range(batch_size):
                # Xử lý caption gốc
                ref_tokens = [vocab.itos[t.item()] for t in captions[j] if t.item() not in [0, 1, 2]]
                refs.append([ref_tokens]) # NLTK cần list of lists cho refs
                
                # Xử lý caption dự đoán
                pred_tokens = [vocab.itos[t.item()] for t in ys[j] if t.item() not in [0, 1, 2]]
                hyps.append(pred_tokens)

    # Tính BLEU
    bleu4 = corpus_bleu(refs, hyps, weights=(0.25, 0.25, 0.25, 0.25))
    print(f"BLEU-4 Score: {bleu4*100:.2f}")

# Gọi hàm đánh giá (lưu ý: chạy hơi lâu vì generation tuần tự)
evaluate_bleu(model, test_loader, device, dataset.vocab)

BLEU-4 Score: 7.00
